# Clase 6

### BERT y Análisis de Sentimientos

En este cuaderno, exploramos el uso de **BERT** (*Bidirectional Encoder Representations from Transformers*), un modelo de lenguaje basado en *transformers* desarrollado por Google. BERT es ampliamente utilizado en tareas de procesamiento de lenguaje natural (NLP), como clasificación de textos, respuestas a preguntas y análisis de sentimientos, gracias a su capacidad para comprender el contexto bidireccional de las palabras en una oración.

###  ¿Qué es BERT?
BERT es un modelo preentrenado que entiende el lenguaje humano a partir de grandes corpus de texto. A diferencia de los enfoques tradicionales, BERT analiza las palabras teniendo en cuenta tanto el contexto previo como el posterior en una oración, lo que mejora significativamente su capacidad para interpretar el significado.

###  ¿Qué hacemos en este cuaderno?
- Cargamos un modelo BERT preentrenado para la tarea de **análisis de sentimientos**.
- Preprocesamos un conjunto de datos que contiene textos y etiquetas de sentimiento (positivo, neutro, negativo).
- Entrenamos el modelo utilizando un dataset en español y evaluamos su rendimiento.
- Realizamos predicciones sobre nuevos textos para determinar su sentimiento.
- Finalmente, analizamos un archivo con comentarios para clasificar automáticamente sus sentimientos.



In [ ]:
# Paso 1: Instalar/Reinstalar las bibliotecas necesarias
!pip install transformers datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 546.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [ ]:
import gdown

print("Descargando dataset...")
url = 'https://drive.google.com/uc?export=download&id=1IRI3NN3wDB3ZPKj7mMNCQTkqF7OYdP5W'
destination = "tweet_eval_sentiment_es.zip"
gdown.download(url, destination, quiet=False)

import zipfile

with zipfile.ZipFile('tweet_eval_sentiment_es.zip', 'r') as zip_ref:
    zip_ref.extractall('')  # Cambia 'ruta_de_extraccion' si quieres un nombre específ

# URL del archivo de texto con comentarios de clientes
url_comentarios = 'https://drive.google.com/uc?export=download&id=12c_JKSe7ijkoMAOpSeJuVWjzfDyh1JIn'
destination = "comentarios.txt"
gdown.download(url_comentarios, destination, quiet=False)

Descargando dataset...


Downloading...
From: https://drive.google.com/uc?export=download&id=1IRI3NN3wDB3ZPKj7mMNCQTkqF7OYdP5W
To: /content/tweet_eval_sentiment_es.zip
100%|██████████| 57.4k/57.4k [00:00<00:00, 3.84MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=12c_JKSe7ijkoMAOpSeJuVWjzfDyh1JIn
To: /content/comentarios.txt
100%|██████████| 1.99k/1.99k [00:00<00:00, 1.59MB/s]


'comentarios.txt'

# Modelo 1

In [ ]:
# Paso 1: Importar las bibliotecas necesarias
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk, Dataset
from collections import Counter
import pandas as pd
import requests

# Deshabilitar wandb
os.environ["WANDB_DISABLED"] = "true"

# Cargar el dataset traducido desde la carpeta
translated_dataset = Dataset.load_from_disk('tweet_eval_sentiment_es')

# Verificar y mostrar los primeros 5 elementos
for idx, item in enumerate(translated_dataset.select(range(5))):
    print(f"Ejemplo {idx + 1}:")
    print(f"Texto: {item['text']}")
    print(f"Etiqueta: {item['label']}")
    print("-" * 40)

# Mostrar la distribución de etiquetas en el dataset completo
label_counts = Counter(translated_dataset['label'])
print("\nDistribución de etiquetas en el dataset:")
print(label_counts)

# Paso 2: Preprocesar los datos
# Cargar el tokenizador del modelo multilingüe
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Función para tokenizar los textos
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Dividir y tokenizar el dataset
train_dataset = translated_dataset.shuffle(seed=42).select(range(800)).map(tokenize_function, batched=True)
test_dataset = translated_dataset.shuffle(seed=42).select(range(800, 1000)).map(tokenize_function, batched=True)

# Establecer el formato de los datos para PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Paso 3: Configurar el modelo y el entrenamiento

# Usar la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo multilingüe pre-entrenado con ignore_mismatched_sizes=True
model = AutoModelForSequenceClassification.from_pretrained(
    'nlptown/bert-base-multilingual-uncased-sentiment',
    num_labels=3,
    ignore_mismatched_sizes=True
).to(device)

# Configurar los argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

# Crear el trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Paso 4: Entrenar el modelo
trainer.train()

# Paso 5: Evaluar el modelo
trainer.evaluate()

# Paso 6: Definir el Mapeo de Etiquetas
label_map = {0: "negativo", 1: "neutro", 2: "positivo"}

# Paso 7: Hacer Inferencias
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return label_map[predicted_class_id]

# Ejemplo de uso
print(predict_sentiment("Este producto es genial!"))
print(predict_sentiment("Este producto es terrible."))

# Leer y analizar los comentarios
with open('comentarios.txt', 'r') as file:
    comentarios = file.readlines()

comentarios_sentimientos = [(comentario.strip(), predict_sentiment(comentario.strip())) for comentario in comentarios]

# Crear un DataFrame con los resultados
df_sentimientos = pd.DataFrame(comentarios_sentimientos, columns=['Comentario', 'Sentimiento'])
print(df_sentimientos)


Ejemplo 1:
Texto: "QT @user En el borrador original del séptimo libro, Remus Lupin sobrevivió a la batalla de Hogwarts. #HappyBirthdayRemusLupin"
Etiqueta: 2
----------------------------------------
Ejemplo 2:
Texto: "Ben Smith / Smith (concusión) permanece fuera de la alineación el jueves, Curtis #NHL #SJ"
Etiqueta: 1
----------------------------------------
Ejemplo 3:
Texto: Lo siento por el arroyo anoche me estrellé pero estaré esta noche seguro, y luego de vuelta a Minecraft en el PC mañana por la noche.
Etiqueta: 1
----------------------------------------
Ejemplo 4:
Texto: El doble RBI de Chase Headley en la octava entrada de David Price rompió una racha de Yankees de 33 entradas consecutivas sin anotación contra Blue Jays
Etiqueta: 1
----------------------------------------
Ejemplo 5:
Texto: @user Alciato: Bee invertirá 150 millones en enero, otros 200 en el verano y planea traer a Messi para 2017"
Etiqueta: 2
----------------------------------------

Distribución de etiquetas en

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.099400
20,1.071600
30,1.034200
40,1.027900
50,0.967600
60,0.981100
70,0.934600
80,0.834400
90,0.838500
100,0.805200


positivo
negativo
                                           Comentario Sentimiento
0   La entrega fue puntual, pero el embalaje podrí...      neutro
1   La atención al cliente fue buena, pero el prod...    negativo
2   El servicio fue adecuado, pero podría mejorar ...      neutro
3   El producto es bueno, aunque esperaba algo más...    positivo
4               Calidad media, pero no me impresionó.      neutro
5   El servicio fue rápido, pero no resolvieron to...      neutro
6   El producto es funcional, pero esperaba algo m...      neutro
7   La entrega fue puntual, pero el embalaje estab...      neutro
8   La calidad es media, cumple con lo básico pero...      neutro
9   El producto es funcional, pero no cumple todas...      neutro
10  El diseño es atractivo, pero no es muy cómodo ...      neutro
11  El precio es razonable, pero la calidad no es ...      neutro
12  El servicio al cliente fue rápido, pero no res...    negativo
13                 El producto cumple con su función.    p

# Agregar datos al dataset tweet_eval_sentiment_es

In [ ]:
# Paso 1: Importar las bibliotecas necesarias
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk, Dataset, concatenate_datasets, Features, Value
from collections import Counter
import pandas as pd
import requests
from sklearn.metrics import classification_report

# Deshabilitar wandb
os.environ["WANDB_DISABLED"] = "true"

# Paso 2: Cargar el dataset traducido desde la carpeta
translated_dataset = load_from_disk('tweet_eval_sentiment_es')


# Mostrar la distribución de etiquetas en el dataset completo
label_counts = Counter(translated_dataset['label'])
print("\nDistribución de etiquetas en el dataset:")
print(label_counts)

# Paso 3: Crear la lista de ejemplos negativos adicionales
extra_negatives = [
    {"text": "No volvería a comprar, mala experiencia.", "label": 0},
    {"text": "El producto llegó dañado, no recomiendo.", "label": 0},
    {"text": "Muy mala calidad, no cumplió mis expectativas.", "label": 0},
    {"text": "El color no coincide con la descripción, decepcionante.", "label": 0},
    {"text": "Servicio al cliente deficiente, no solucionaron mi problema.", "label": 0}
]

# Obtener el esquema de características del dataset original
label_feature = translated_dataset.features['label']

# Definir las características para el nuevo dataset
features = Features({
    'text': Value('string'),
    'label': label_feature
})

# Crear el dataset adicional con las características especificadas
extra_negatives_dataset = Dataset.from_list(extra_negatives, features=features)

# Combinar el dataset traducido con los ejemplos negativos adicionales
augmented_dataset = concatenate_datasets([translated_dataset, extra_negatives_dataset])

# Mostrar la distribución de etiquetas en el dataset aumentado
label_counts_augmented = Counter(augmented_dataset['label'])
print("\nDistribución de etiquetas en el dataset aumentado:")
print(label_counts_augmented,"\n")
print("-"*100)
dataset_size = len(augmented_dataset)

# Verificar y mostrar los primeros 5 elementos
for idx, item in enumerate(augmented_dataset.select(range(5))):
    print(f"Ejemplo {idx + 1}:")
    print(f"Texto: {item['text']}")
    print(f"Etiqueta: {item['label']}")
    print("-" * 40)




Distribución de etiquetas en el dataset:
Counter({1: 460, 2: 389, 0: 151})

Distribución de etiquetas en el dataset aumentado:
Counter({1: 460, 2: 389, 0: 156}) 

----------------------------------------------------------------------------------------------------
Ejemplo 1:
Texto: "QT @user En el borrador original del séptimo libro, Remus Lupin sobrevivió a la batalla de Hogwarts. #HappyBirthdayRemusLupin"
Etiqueta: 2
----------------------------------------
Ejemplo 2:
Texto: "Ben Smith / Smith (concusión) permanece fuera de la alineación el jueves, Curtis #NHL #SJ"
Etiqueta: 1
----------------------------------------
Ejemplo 3:
Texto: Lo siento por el arroyo anoche me estrellé pero estaré esta noche seguro, y luego de vuelta a Minecraft en el PC mañana por la noche.
Etiqueta: 1
----------------------------------------
Ejemplo 4:
Texto: El doble RBI de Chase Headley en la octava entrada de David Price rompió una racha de Yankees de 33 entradas consecutivas sin anotación contra Blue Ja

In [ ]:
# TODO 1: Añadir más ejemplos negativos, positivos y neutros para equilibrar aún más el dataset.

# TODO 2: Combinar el dataset con los ejemplos adicionales

# TODO 3: Entrenar con el dataset aumentado

# TODO 4: Evaluar el modelo usando una métrica adicional, como f1-score, ROC-AUC, o la que considere adecuada.

# TODO 5: Modificar hiperparámetros como lr, batch_size, num_epochs y otros que considere necesario, para optimizar el modelo.

# TODO 6: Implementar early stopping para evitar sobreajuste.


In [ ]:
# complementario
# Probar con un modelo pre-entrenado diferente, por ejemplo, 'dccuchile/bert-base-spanish-wwm-cased'.